In [1]:
import boto3
import pandas as pd
from io import StringIO

In [ ]:
AWS_ACCESS_KEY= " "
AWS_SECRET_KEY = " "
AWS_REGION = " "
SCHEMA_NAME = " "
S3_STAGING_DIR = " " #OUTPUT FOLDER
S3_BUCKET_AME = " "
S3_OUTPUT_DIRECTORY = " "

In [ ]:
athena_client = boto3.client(
    "athena",
    aws_access_key_id = AWS_ACCESS_KEY,
    aws_secret_access_key = AWS_SECRET_KEY,
    region_name = AWS_REGION, 
)

In [ ]:
Dict = {}
def download_and_load_query_results(client: boto3.client, query_response:Dict)->pd.DataFrame:
    while True:
        try:
            #only loads first 1000 rows
            client.get_query_results(
            QueryExecutionId = query_response["QueryExcutionId"]
            )
            break
        except Exception as err:
            if "not yet finished" in str(err):
                time.sleep(0.001)
            else:
                raise err
    temp_file_location: str ="athena_query_results.csv"
    s3_client = boto3.client(
        "s3",
        aws_access_key_id = AWS_ACCESS_KEY,
        aws_secret_access_key = AWS_SECRET_KEY,
        region_name = AWS_REGION, 
    )
    s3_client.download_file(
        S3_BUCKET_NAME, 
        f"{S3_OUTPUT_DIRECTORY}/{query_response['QueryExecutionId']}.csv",
        temp_file_location,
    )
    return pd.read_csv(temp_file_location)

In [ ]:
response = athena_client.start_query_execution(
    QueryString = "SELECT * FROM enigma_jhud",
    QueryExecutionContext = {"Database": SCHEMA_NAME},
    ResultConfiguration = {
        "OutputLocation" : S3_STAGING_DIR,
        "EncryptionConfiguration" :{"EncryptionOption": "SSE_S3"},
    },

)

In [ ]:
enigma_jhud = download_and_load_query_results(athena_client, response )

In [ ]:
enigma_jhud.head()

In [ ]:
response = athena_client.start_query_execution(
    QueryString = "SELECT * FROM nytimes_data_in_usa_us_states",
    QueryExecutionContext = {"Database": SCHEMA_NAME},
    ResultConfiguration = {
        "OutputLocation" : S3_STAGING_DIR,
        "EncryptionConfiguration" :{"EncryptionOption": "SSE_S3"},
    },

)
nytimes_data_in_usa_us_states = download_and_load_query_results(athena_client, response)


In [ ]:
response = athena_client.start_query_execution(
    QueryString = "SELECT * FROM static_datacountrycode",
    QueryExecutionContext = {"Database": SCHEMA_NAME},
    ResultConfiguration = {
        "OutputLocation" : S3_STAGING_DIR,
        "EncryptionConfiguration" :{"EncryptionOption": "SSE_S3"},
    },

)
static_datacountrycode = download_and_load_query_results(athena_client, response)


In [ ]:
response = athena_client.start_query_execution(
    QueryString = "SELECT * FROM static_datacountypopulation",
    QueryExecutionContext = {"Database": SCHEMA_NAME},
    ResultConfiguration = {
        "OutputLocation" : S3_STAGING_DIR,
        "EncryptionConfiguration" :{"EncryptionOption": "SSE_S3"},
    },

)
static_datacountypopulation = download_and_load_query_results(athena_client, response)


In [ ]:
response = athena_client.start_query_execution(
    QueryString = "SELECT * FROM rearc_usa_hospital_beds",
    QueryExecutionContext = {"Database": SCHEMA_NAME},
    ResultConfiguration = {
        "OutputLocation" : S3_STAGING_DIR,
        "EncryptionConfiguration" :{"EncryptionOption": "SSE_S3"},
    },

)
rearc_usa_hospital_beds = download_and_load_query_results(athena_client, response)


In [ ]:
response = athena_client.start_query_execution(
    QueryString = "SELECT * FROM rearc_covid_19_testing_dataus_total_test ",
    QueryExecutionContext = {"Database": SCHEMA_NAME},
    ResultConfiguration = {
        "OutputLocation" : S3_STAGING_DIR,
        "EncryptionConfiguration" :{"EncryptionOption": "SSE_S3"},
    },

)
rearc_covid_19_testing_dataus_total_test = download_and_load_query_results(athena_client, response)


In [ ]:
response = athena_client.start_query_execution(
    QueryString = "SELECT * FROM rearc_covid_19_testing_data_states_dailystates_daily ",
    QueryExecutionContext = {"Database": SCHEMA_NAME},
    ResultConfiguration = {
        "OutputLocation" : S3_STAGING_DIR,
        "EncryptionConfiguration" :{"EncryptionOption": "SSE_S3"},
    },

)
rearc_covid_19_testing_data_states_dailystates_daily = download_and_load_query_results(athena_client, response)


In [ ]:
static_datastate_abv.head()

In [ ]:
#Changing the headers
new_header = static_datastate_abv.iloc[0]

In [ ]:
static_datastate_abv = static_datastate_abv[1:]

In [ ]:
static_datastate_abv.columns = new_header

In [ ]:
static_datastate_abv.head()

In [ ]:
#Star schema , creating fact table and dimension tables

factcovid1 = enigma_jhud[["fips","province_state","country_region","confirmed","deaths","recovered","active"]]
factcovid2 = rearc_covid_19_testing_data_states_dailystates_daily[["fips","date","positive","negative","hospitalizedcurrently","hospitalized","hospitalizedischarged"]]

factcovid = pd.merge(factcovid1, factcovid2, on= 'fips',how="inner")

In [ ]:
factcovid.head()

In [ ]:
dimregion1 = enigma_jhud[["fips","province_state","country_region","confirmed","deaths","recovered","active"]]
dimregion2 = nytimes_data_in_usa_us_country[["fips","country","state"]]

dimregion = pd.merge(dimregion1,dimregion2,on='fips',how='inner')

In [ ]:
dimregion.head()

In [ ]:
dimdate = rearc_covid_19_testing_data_states_dailystates_daily[["fips","date"]]
dimdate.head()

In [ ]:
dimdate['date'] = pd.to_datetime(dimDate['date'],format ="&Y%m%d")


In [ ]:
dimdate['year'] =dimdate['date'].dt.year

dimdate['month']  = dimdate['date'].dt.month

dimdate['day_of_week'] = dimdate['date'].dt.dayofweek 

In [ ]:
dimdate.head()

In [ ]:
#storing results in s3

bucket = "covid-project-23" #already created in s3


In [ ]:
csv_buffer = StringIO()
factcovid.to_csv(csv_buffer)
s3_resource = boto3.resource('s3')

s3_resource.object(bucket,"output/factcovid.csv").put(Body =csv_buffer.getvalue())

In [ ]:
dimdatesql = pd.io.sql.get_schema(dimdate.reset_index(),'dimDate')
print(''.join(dimdatesql))

In [ ]:
factcovidsql = pd.io.sql.get_schema(factcovid.reset_index(),'factcovid')
print(''.join(factcovidsql))

In [ ]:
dimregionsql = pd.io.sql.get_schema(dimregion.reset_index(),'dimregion')
print(''.join(dimregionsql))

In [ ]:
import redshift_connector

In [ ]:
conn = redshift_connector.connect(
    host = 'redshift-cluster-2.ctlwvzbuur6m.ap-south-1.redshift.amazonaws.com',
    database = 'dev',
    user = 'awsuser',
    password = 'Passw0rd123'
)

In [ ]:
conn.autocommit = True

In [ ]:
cursor = redshift_connector.cursor = conn.cursor()

In [ ]:
cursor.execute("""CREATE TABLE dimdate(
    "index" INTEGER,
    "fips" INTEGER,
    "date" TIMESTAMP,
    "year" INTEGER,
    "month" INTEGER,
    "day_of_week" INTEGER
)""")


cursor.execute("""
    CREATE TABLE "dimhospital"(
    "index" INTEGER,
    "fips" REAL,
    "state_name" TEXT,
    "latitude" REAL,
    "longitude" TEXT,
    "hq_address" TEXT,
    "hospital_name" TEXT,
    "hospital_type" TEXT,
    "hq_city" TEXT,
    "hq_state" TEXT
    )

""")


cursor.execute("""
    CREATE TABLE "factcovid"(
    "index" INTEGER,
    "fips" REAL,
    "province_state" TEXT,
    "country_region" TEXT,
    "confirmed" REAL,
    "deaths" REAL,
    "recovered" REAL,
    "active" REAL, 
    "date" INTEGER,
    "positive" REAL,
    "negative"REAL ,
    "hospitalizedcurrently" REAL,
    "hospitalized" REAL,
    "hospitalizeddischarged" REAL
    )

""")
